# Predicting Nursing Home Fines

[Direct Supply's](https://www.directsupply.com/) business is focused on senior living and healthcare. We have services in procurement, building management, design and construction, and health and wellness. The ability to predict fines a nursing home might receive would have several benefits to both the nursing home and to Direct Supply. Direct Supply may be able to help the nursing home avoid fines through the products and services that we offer. We can focus sales and engagement efforts around the things we know they need. From Direct Supply's perspective, we may be able to anticipate a downturn in sales, better estimate credit worthiness, or get ahead of potential merger and acquisition activity.

# Exploring the Data

* Identify categorical and numeric features
* Visualize the data

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer, KBinsDiscretizer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

pd.options.mode.chained_assignment = None
pd.options.display.max_rows = 500
pd.options.display.max_columns = 500
pd.options.display.width = 1000

# NOTE: This may not be necessary in future versions of scikit-learn, but it is
#       necessary here to get the feature names out of the ColumnTransformer.
#       All of the other transformers support this method.
SimpleImputer.get_feature_names_out = (
    lambda self, names=None: self.feature_names_in_
)
plt.style.use('seaborn-dark')

We'll start with importing the master _Provider Information_ dataset from the US government's CMS website. This inlcudes a wealth of columns that we'll encode to build our base model. It also includes the column "Total Amount of Fines in Dollars", which is what our prediction target will be.

In [ ]:
provider_info_df = pd.read_csv('NH_ProviderInfo_Jan2022.csv')
provider_info_df.info()

In [ ]:
provider_info_df.head()

Let's take a quick look at the mathematical summary of the prediction target, or the total fine amount per facility.

In [ ]:
provider_info_df['Total Amount of Fines in Dollars'].describe()

For the purposes of train-test splitting, we'll want to discretize the total fine amount into a small amount of buckets such that we can use that value to pass to the _stratify_ parameter on scikit-learn's `train_test_split` function.

In [ ]:
sns.histplot(provider_info_df['Total Amount of Fines in Dollars'], bins=10)
plt.title('Total Amount of Fines in Dollars', fontsize=15)

It appears that we may want to remove outliers > $500,000 or so, later, when we improve the model.

Our dataset has many interesting columns. From inspecting them, we've identified the following numerical and categorical features:

In [ ]:
num_cols = [
    'Number of Certified Beds', 'Average Number of Residents per Day', 'Overall Rating', 'Health Inspection Rating', 'QM Rating', 'Long-Stay QM Rating', 
    'Short-Stay QM Rating', 'Staffing Rating', 'RN Staffing Rating', 'Reported Nurse Aide Staffing Hours per Resident per Day',
    'Reported LPN Staffing Hours per Resident per Day', 'Reported RN Staffing Hours per Resident per Day', 'Reported Licensed Staffing Hours per Resident per Day',
    'Reported Total Nurse Staffing Hours per Resident per Day', 'Total number of nurse staff hours per resident per day on the weekend',
    'Registered Nurse hours per resident per day on the weekend', 'Reported Physical Therapist Staffing Hours per Resident Per Day',
    'Total nursing staff turnover', 'Registered Nurse turnover',  'Number of administrators who have left the nursing home',
    'Case-Mix Nurse Aide Staffing Hours per Resident per Day', 'Case-Mix LPN Staffing Hours per Resident per Day', 'Case-Mix RN Staffing Hours per Resident per Day',
    'Case-Mix Total Nurse Staffing Hours per Resident per Day', 'Adjusted Nurse Aide Staffing Hours per Resident per Day',
    'Adjusted LPN Staffing Hours per Resident per Day', 'Adjusted RN Staffing Hours per Resident per Day', 'Adjusted Total Nurse Staffing Hours per Resident per Day',
    'Rating Cycle 1 Total Number of Health Deficiencies', 'Rating Cycle 1 Number of Standard Health Deficiencies',
    'Rating Cycle 1 Number of Complaint Health Deficiencies', 'Rating Cycle 1 Health Deficiency Score', 'Rating Cycle 1 Number of Health Revisits',
    'Rating Cycle 1 Health Revisit Score', 'Rating Cycle 1 Total Health Score', 'Rating Cycle 2 Total Number of Health Deficiencies',
    'Rating Cycle 2 Number of Standard Health Deficiencies', 'Rating Cycle 2 Number of Complaint Health Deficiencies',
    'Rating Cycle 2 Health Deficiency Score', 'Rating Cycle 2 Number of Health Revisits', 'Rating Cycle 2 Health Revisit Score',
    'Rating Cycle 2 Total Health Score', 'Rating Cycle 3 Total Number of Health Deficiencies', 'Rating Cycle 3 Number of Standard Health Deficiencies',
    'Rating Cycle 3 Number of Complaint Health Deficiencies', 'Rating Cycle 3 Health Deficiency Score', 'Rating Cycle 3 Number of Health Revisits',
    'Rating Cycle 3 Health Revisit Score', 'Rating Cycle 3 Total Health Score', 'Total Weighted Health Survey Score', 'Number of Facility Reported Incidents',
    'Number of Substantiated Complaints', 'Number of Citations from Infection Control Inspections']

In [ ]:
cat_cols = [
    'Provider City', 'Provider State', 'Provider Zip Code', 'Provider SSA County Code', 'Ownership Type', 'Provider Type', 'Provider Resides in Hospital', 
    'Continuing Care Retirement Community', 'Special Focus Status', 'Abuse Icon', 'Most Recent Health Inspection More Than 2 Years Ago', 
    'Provider Changed Ownership in Last 12 Months', 'With a Resident and Family Council', 'Automatic Sprinkler Systems in All Required Areas',
    'Long-Stay QM Rating Footnote', 'Short-Stay QM Rating Footnote', 'Staffing Rating Footnote', 'RN Staffing Rating Footnote', 'Reported Staffing Footnote', 
    'Physical Therapist Staffing Footnote', 'Total nursing staff turnover footnote', 'Registered Nurse turnover footnote', 'Administrator turnover footnote']
for cat_col in cat_cols:
    provider_info_df[cat_col] = provider_info_df[cat_col].astype('category')

# Creating a Baseline Model

* Minimal feature engineering
* No external datasets
* Linear model

In [ ]:
# Transforms categorical, numerical, and binned features using imputation, scaling, and encoding
# Fits the estimator with the training set and returns predicted labels for the X_test set
# Train/test splitting and error computation is left to the caller
def run_baseline_model(X_train, X_test, y_train, estimator, categorical_features=[], numerical_features=[], binned_features=[], n_bins=15):
  numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
  ])

  categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
  ])

  binning_transformer = Pipeline(steps=[
    ('encoder', KBinsDiscretizer(strategy='uniform', n_bins=n_bins))
  ])

  ct = ColumnTransformer(
      remainder='drop',
      transformers=[
        ('numerical', numeric_transformer, numerical_features),
        ('categorical', categorical_transformer, categorical_features),
        ('binned', binning_transformer, binned_features)
      ]
  )

  # First we fit the transformers to the training set and transform it
  X_train_prepared = pd.DataFrame(
    ct.fit_transform(X_train).toarray(),
    columns=ct.get_feature_names_out())
  
  # Then we only transform the test set to make use of the imputation, scaling, 
  # and categorization parameters from the training set.
  X_test_prepared = pd.DataFrame(
      ct.transform(X_test).toarray(),
      columns=ct.get_feature_names_out())

  estimator.fit(X_train_prepared, y_train)
  y_pred = estimator.predict(X_test_prepared)

  return y_pred

In [ ]:
fines = provider_info_df[['Total Amount of Fines in Dollars']]
discretizer = KBinsDiscretizer(n_bins=7, encode='ordinal', strategy='quantile')
binned_fines = discretizer.fit_transform(fines)
sns.histplot(binned_fines)
plt.title('Binned Fines', fontsize=15)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    provider_info_df, provider_info_df['Total Amount of Fines in Dollars'], 
    train_size=.75, stratify=binned_fines)

y_pred = run_baseline_model(X_train, X_test, y_train, ElasticNet(positive=True), cat_cols, num_cols)

In [ ]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"RMSE: ${rmse:.2f}")

In [ ]:
def plot_actual_vs_predicted(y_test, y_pred, xlabel='Actual Fines (\$)', ylabel='Predicted Fines (\$)'):
    fig = plt.figure(figsize=(5, 5))
    sns.scatterplot(x=y_test, y=y_pred)
    
    all_values = np.concatenate((y_test, y_pred))
    x = np.linspace(np.min(all_values), np.max(all_values))
    y = x

    plt.plot(x, y, color='orange')
    plt.title(f"{xlabel} vs {ylabel}", fontsize=15)
    plt.xlabel(xlabel, fontsize=10)
    plt.ylabel(ylabel, fontsize=10)
    plt.show()

In [ ]:
plot_actual_vs_predicted(y_test, y_pred)

# Enhance the Model

## Additional Feature Engineering

Up next, we'll look into our existing column set and see if we can engineer more features from them that may improve the model.

### Geocoding

In a separate notebook, we used the `geopy`'s geocoding capabilities to reach out to a combination of web services (namely [Nominatim](https://nominatim.org/) and [Bing Maps](https://www.bingmapsportal.com/)) to geocode the addresses of each facility. That is, each address was converted to a latitude and longitude, which were then appended to the dataset as additional columns, and exported to a new CSV file. Let's import that dataset and re-run the experiment.

In [ ]:
enhanced_model_df = pd.read_csv('NH_ProviderInfo_Jan2022_with_coords.csv').drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])
enhanced_model_df.head()

We'll add the latitude and longitude as numeric columns, which will be scaled with the `StandardScaler`.

In [ ]:
enhanced_num_cols = num_cols + ['lat', 'lon']
enhanced_model_df.info()

Let's run with the coordinates.

In [ ]:
fines = enhanced_model_df[['Total Amount of Fines in Dollars']]
discretizer = KBinsDiscretizer(n_bins=7, encode='ordinal', strategy='quantile')
binned_fines = discretizer.fit_transform(fines)
sns.histplot(binned_fines)
plt.title('Binned Fines', fontsize=15)

In [ ]:
fine_amount_feature = 'Total Amount of Fines in Dollars'

X_train, X_test, y_train, y_test = train_test_split(
    enhanced_model_df, enhanced_model_df[fine_amount_feature], 
    train_size=.75, stratify=binned_fines)

y_pred = run_baseline_model(X_train, X_test, y_train, ElasticNet(positive=True), cat_cols, enhanced_num_cols)

In [ ]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"RMSE: ${rmse:.2f}")

## Remove Outliers

In [ ]:
no_outliers_df = enhanced_model_df.copy()
no_outliers_df = no_outliers_df[enhanced_model_df[fine_amount_feature] < no_outliers_df[fine_amount_feature].quantile(0.99)]
no_outliers_df = no_outliers_df[enhanced_model_df[fine_amount_feature] > no_outliers_df[fine_amount_feature].quantile(0.01)]
no_outliers_df.shape

In [ ]:
fines = no_outliers_df[['Total Amount of Fines in Dollars']]
discretizer = KBinsDiscretizer(n_bins=7, encode='ordinal', strategy='quantile')
binned_fines = discretizer.fit_transform(fines)
sns.histplot(binned_fines)
plt.title('Binned Fines', fontsize=15)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    no_outliers_df, no_outliers_df[fine_amount_feature], 
    train_size=.75, stratify=binned_fines)

y_pred = run_baseline_model(X_train, X_test, y_train, ElasticNet(positive=True), cat_cols, enhanced_num_cols)

In [ ]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"RMSE: ${rmse:.2f}")

This is decent improvement so far.

In [ ]:
plot_actual_vs_predicted(y_test, y_pred)

## Additional Datasets

Join in state averages and quality MSR data. _Note_ that the MSR set will need to be grouped by the `Federal Provider Number`.

### State Averages

In [ ]:
state_averages_df = pd.read_csv('NH_StateUSAverages_Jan2022.csv')
state_averages_df.head()

In [ ]:
state_averages_df.columns = state_averages_df.columns.map(lambda x: str(x) + '_State')
state_averages_df = state_averages_df.rename(columns={
    'State or Nation_State': 'Provider State'
}, errors='ignore')
state_averages_df.info()

In [ ]:
additional_data_df = no_outliers_df.merge(state_averages_df, on='Provider State', how='left')
additional_data_df.head()

In [ ]:
new_fields = [x for x in state_averages_df.columns if x not in ['Provider State', 'Processing Date_State', 'Number of Fines_State']]
additional_data_num_cols = enhanced_num_cols + new_fields
print(len(additional_data_num_cols))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    additional_data_df, additional_data_df[fine_amount_feature], 
    train_size=.75, stratify=binned_fines)

y_pred = run_baseline_model(X_train, X_test, y_train, ElasticNet(positive=True), cat_cols, additional_data_num_cols)

In [ ]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"RMSE: ${rmse:.2f}")

In [ ]:
plot_actual_vs_predicted(y_test, y_pred)

Not much improvement from bringing in state averages.

### Next dataset

In [ ]:
quality_mds_df = pd.read_csv('NH_QualityMsr_MDS_Jan2022.csv')
quality_mds_df['Federal Provider Number'] = quality_mds_df['Federal Provider Number'].map(lambda x: str(x)[1:] if str(x)[0] == '0' else str(x))

In [ ]:
footnote_cols = [x for x in quality_mds_df.columns if x.startswith('Footnote')]
quality_mds_df = quality_mds_df.drop(columns=['Provider Name', 'Provider Address', 'Provider City', 'Provider State', 'Provider Zip Code', 'Location', 'Processing Date', 'Measure Period', 'Measure Description', 'Measure Code'] + footnote_cols, errors='ignore')
quality_mds_df.head()

In [ ]:
quality_mds_agg = quality_mds_df.groupby(['Federal Provider Number']).mean()
quality_mds_agg.head()

In [ ]:
quality_mds_agg.columns = quality_mds_agg.columns.map(lambda x: str(x) + '_mds')
quality_mds_agg.head()

In [ ]:
data_with_mds_df = additional_data_df.merge(quality_mds_agg, on='Federal Provider Number', how='left')
data_with_mds_df.head()

In [ ]:
data_with_mds_df.describe()

In [ ]:
new_cols = ['Q1 Measure Score_mds', 'Q2 Measure Score_mds', 'Q3 Measure Score_mds', 'Q4 Measure Score_mds', 'Four Quarter Average Score_mds']
num_cols_with_mds = additional_data_num_cols + new_cols

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data_with_mds_df, data_with_mds_df[fine_amount_feature], 
    train_size=.75, stratify=binned_fines)

y_pred = run_baseline_model(X_train, X_test, y_train, ElasticNet(positive=True), cat_cols, num_cols_with_mds)

In [ ]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"RMSE: ${rmse:.2f}")

In [ ]:
plot_actual_vs_predicted(y_test, y_pred)

## More Complex Model Types

Perhaps we can use some cross validation here to select a model or use grid search to tune model hyperparameters.

# Summary of Model Improvements

# Minimal Model for Explanatory Purposes

Perhaps use a correlation matrix to find features that don't provide much and remove them.